In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv(r"D:\SNU\4th sem\DESIGN AND ANALYSIS OF ALGORITHMS\CIA2\Bank_Personal_Loan_Modelling.csv")

In [3]:
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [4]:
df.drop(['ID','ZIP Code'],axis=1,inplace=True)

In [5]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [6]:
df=df[['Age', 'Experience', 'Income','Family','CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard','Personal Loan']]

In [7]:
x= df.iloc[:,:-1].values
y= df.iloc[:, -1].values

In [8]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [10]:
from keras.models import Sequential
from keras.layers import Dense

In [12]:
model=Sequential()
model.add(Dense(units=6,activation="relu"))
model.add(Dense(units=6,activation="relu"))#second hidden layer is created
model.add(Dense(units=1,activation="sigmoid"))#another layer with different activation fn(o/p layer))

In [13]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics="accuracy")

In [19]:
def formPheremoneMatrix(model):
    nv = len(model).astype(bool)
    pheremones = np.ones((nv,nv))
    return pheremones

In [20]:
def chooseNextVertex(model, pheremones, currPos):
    denominator = np.dot(model[currPos], pheremones[currPos])
    numerator = model[currPos] * pheremones[currPos]
    
    probabilities = numerator/denominator
    
    
    rouletteWheel = np.cumsum(probabilities)
    
    rouletteBall = np.random.random()
    
    nextVertex = np.where(rouletteWheel >= rouletteBall)[0][0]
    
    return nextVertex

In [21]:
def traverse(model, pheremones, start, end):
    curr = start
    path = [curr]
    cost = 0
    prev = start
    
    while curr != end:
        nextVertex = chooseNextVertex(model, pheremones, curr)
        
        while nextVertex == prev:
            nextVertex = chooseNextVertex(model, pheremones, curr)
        
        cost += model[curr][nextVertex]
        path += [nextVertex]
        prev = curr
        curr = nextVertex
        
    
    return path, cost

In [22]:

def releaseGeneration(model, pheremones, start, end, size = 10):
    paths = []
    costs = []
    for i in range(size):
        p, c = traverse(model, pheremones, start, end)
        paths += [p]
        costs += [c]
    costs = np.array(costs)    
    
    return paths, costs

In [23]:
def updatePheremones(paths, costs, pheremones, decay = 0):
    pheremones = (1-decay)*pheremones
    
    costs = 1/costs
    for p in range(len(paths)):
        path = paths[p]
        for i in range(len(path) - 1):
            pheremones[path[i]][path[i+1]] += costs[p]
    
    return pheremones

In [24]:
model.fit(x_train, y_train, batch_size = 30, epochs = 100)

Epoch 1/100
125/125 [==============================] - 1s 2ms/step - loss: 0.5467 - accuracy: 0.8571
Epoch 2/100
125/125 [==============================] - 0s 2ms/step - loss: 0.3269 - accuracy: 0.9008
Epoch 3/100
125/125 [==============================] - 0s 2ms/step - loss: 0.2341 - accuracy: 0.9008
Epoch 4/100
125/125 [==============================] - 0s 2ms/step - loss: 0.2033 - accuracy: 0.9008
Epoch 5/100
125/125 [==============================] - 0s 2ms/step - loss: 0.1880 - accuracy: 0.9008
Epoch 6/100
125/125 [==============================] - 0s 2ms/step - loss: 0.1781 - accuracy: 0.9008
Epoch 7/100
125/125 [==============================] - 0s 2ms/step - loss: 0.1700 - accuracy: 0.9011
Epoch 8/100
125/125 [==============================] - 0s 2ms/step - loss: 0.1627 - accuracy: 0.9125
Epoch 9/100
125/125 [==============================] - 0s 2ms/step - loss: 0.1544 - accuracy: 0.9277
Epoch 10/100
125/125 [==============================] - 0s 2ms/step - loss: 0.1439 - accura

In [26]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_test, y_pred)

array([[1134,    8],
       [  17,   91]], dtype=int64)

In [27]:
accuracy_score(y_test, y_pred)

0.98